In [1]:
import setup
setup.init()

from aiproject.prettifier import pprint
from ai.agents import get_document_agent, get_movies_agent
from langgraph.checkpoint.memory import InMemorySaver
from ai.supervisors import get_supervisor
checkpointer = InMemorySaver()
doc_agent = get_document_agent(checkpointer)
mov_agent = get_movies_agent(checkpointer)
supervisor = get_supervisor(checkpointer)

In [2]:
from mypermit import permit_client
permit = permit_client

In [3]:
await permit.api.users.list()

PaginatedResultUserRead(data=[UserRead(key='3', id=UUID('0cb9667e-9db4-4304-96d7-5124d68f2df2'), organization_id=UUID('715045d3-e31a-4694-b9b2-99e92ba3710e'), project_id=UUID('5da7dcbf-5782-4c70-ac12-b3bf4ba17ebf'), environment_id=UUID('41c7b873-45dc-4fda-a372-7a3a2d6131df'), associated_tenants=[UserInTenant(tenant='default', roles=['manager'], status=<UserStatus.pending: 'pending'>, resource_instance_roles=[])], roles=[UserRole(role='manager', tenant='default')], created_at=datetime.datetime(2025, 11, 7, 23, 42, 4, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2025, 11, 7, 23, 45, 52, tzinfo=datetime.timezone.utc), email=None, first_name='abdullah', last_name=None, attributes=None), UserRead(key='2', id=UUID('a8fdd435-4899-4b38-9932-1bfba6c72ec4'), organization_id=UUID('715045d3-e31a-4694-b9b2-99e92ba3710e'), project_id=UUID('5da7dcbf-5782-4c70-ac12-b3bf4ba17ebf'), environment_id=UUID('41c7b873-45dc-4fda-a372-7a3a2d6131df'), associated_tenants=[UserInTenant(tenant='defau

In [4]:
from django.contrib.auth import get_user_model

User = get_user_model()

In [5]:
for u in User.objects.all():
    print(u.id, u.username)
    user_data = {
        "key": f"{u.id}",
        "first_name": f"{u.username}"
    }
    response = await permit.api.users.sync(user_data)
    print(response)

1 admin
key='1' id=UUID('ae97b250-e6ce-44ab-a5a0-40f04b33217e') organization_id=UUID('715045d3-e31a-4694-b9b2-99e92ba3710e') project_id=UUID('5da7dcbf-5782-4c70-ac12-b3bf4ba17ebf') environment_id=UUID('41c7b873-45dc-4fda-a372-7a3a2d6131df') associated_tenants=[UserInTenant(tenant='default', roles=['manager'], status=<UserStatus.pending: 'pending'>, resource_instance_roles=[])] roles=[UserRole(role='manager', tenant='default')] created_at=datetime.datetime(2025, 11, 7, 23, 42, 2, tzinfo=datetime.timezone.utc) updated_at=datetime.datetime(2025, 11, 7, 23, 45, 51, tzinfo=datetime.timezone.utc) email=None first_name='admin' last_name=None attributes=None
2 staff
key='2' id=UUID('a8fdd435-4899-4b38-9932-1bfba6c72ec4') organization_id=UUID('715045d3-e31a-4694-b9b2-99e92ba3710e') project_id=UUID('5da7dcbf-5782-4c70-ac12-b3bf4ba17ebf') environment_id=UUID('41c7b873-45dc-4fda-a372-7a3a2d6131df') associated_tenants=[UserInTenant(tenant='default', roles=['viewer'], status=<UserStatus.pending: 'pe

In [6]:
res = await permit.check("1", "read", "document")
print(res)

True


In [7]:
resources = [
    {
        "name": 'Document',
        "key": "document",
        "actions": {
            "create": {},
            "read": {},
            "update": {},
            "delete": {}
        }
    },
    {
        "name": "Movie Discovery",
        "key": "movie_discovery",
        "actions": {
            "search": {},
            "detail": {},
        }
    }
]

In [8]:
from permit import PermitApiError

async def get_or_create_resource(resource_data: dict):
    resource_key = resource_data.get('key')
    try:
        _resource = await permit.api.resources.get(resource_key)
    except PermitApiError as e:
        if e.status_code == 404:
            _resource = await permit.api.resources.create(resource_data)
        else:
            raise e
    
    return _resource



In [9]:
roles = [
    {
        "name": "Manager",
        "key": "manager",
        "permissions": [
            "document:create",
            "document:read",
            "document:update",
            "document:delete",
            "movie_discovery:search",
            "movie_discovery:detail",
        ]
        },
        {
        "name": "Viewer",
        "key": "viewer",
        "permissions": [
            "document:read",
            "movie_discovery:search",
            "movie_discovery:detail",
        ],
    }
]

In [10]:
viewer_role = await permit.api.roles.get("viewer")
viewer_role.permissions

['movie_discovery:detail', 'document:read', 'movie_discovery:search']

In [11]:
for user in User.objects.all():
    print(user)

await permit.check("1", "create", "document")

admin
staff
abdullah
weak_user


True

In [12]:
from permit import PermitApiError

async def get_or_create_role(role_data: dict):
    _key = role_data.get('key')
    try:
        _resource = await permit.api.roles.get(_key)
    except PermitApiError as e:
        if e.status_code == 404:
            _resource = await permit.api.roles.create(role_data)
        else:
            raise e
    
    return _resource



In [13]:
for role in roles:
    print(role)
    response = await get_or_create_role(role)
    print(response)

{'name': 'Manager', 'key': 'manager', 'permissions': ['document:create', 'document:read', 'document:update', 'document:delete', 'movie_discovery:search', 'movie_discovery:detail']}
name='Manager' description=None permissions=['document:delete', 'document:read', 'document:create', 'document:update', 'movie_discovery:search', 'movie_discovery:detail'] attributes={} extends=[] granted_to=None v1compat_settings=None v1compat_attributes=None key='manager' id=UUID('3fcbe878-53ac-4824-af27-19b076b4223d') organization_id=UUID('715045d3-e31a-4694-b9b2-99e92ba3710e') project_id=UUID('5da7dcbf-5782-4c70-ac12-b3bf4ba17ebf') environment_id=UUID('41c7b873-45dc-4fda-a372-7a3a2d6131df') created_at=datetime.datetime(2025, 11, 8, 0, 16, 25, tzinfo=datetime.timezone.utc) updated_at=datetime.datetime(2025, 11, 8, 0, 16, 25, tzinfo=datetime.timezone.utc)
{'name': 'Viewer', 'key': 'viewer', 'permissions': ['document:read', 'movie_discovery:search', 'movie_discovery:detail']}
name='Viewer' description=None p

In [14]:
user = User.objects.all().last()
user.id

4

In [16]:
import uuid

config = {"configurable": {
    "user_id": '4',
    "thread_id": uuid.uuid4(),
    }
}

In [ ]:
# Test with user who doesn't have permissions
x = supervisor.invoke(
    {"messages": [{
        "role": 'user',
        "content": "show me content of abdu(3)"
    }]},
    config,
)
pprint(x['messages'][-1].content)

In [ ]:
await permit.check(f"{user.id}", "read", "document")

True

In [ ]:
readonly_permissions = [
    "documents:read",
    "movie_discovery:search",
    "movie_discovery:detail",
]